In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision ## Contains some utilities for working with the image data
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
#%matplotlib inline
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
dataset = MNIST(root = 'data/', download = True)
print(len(dataset))

60000


In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
import pickle

In [ ]:
with open('mnist.pickle', 'rb') as f:
    loaded_dataset = pickle.load(f)
# Assuming the loaded data has 'data' and 'targets' attributes
data = loaded_dataset.data
labels = loaded_dataset.targets

print(len(data), len(labels))
print(type(data[0]))
print(labels[0])
print(max(labels), min(labels))

60000 60000
<class 'torch.Tensor'>
tensor(5)
tensor(9) tensor(0)


In [ ]:
import torchvision.transforms as transforms

train_data = MNIST(root='data/', train=True, download=True, transform=transforms.ToTensor())
test_data = MNIST(root='data/', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=10, shuffle=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([10, 1, 28, 28])
Shape of y: torch.Size([10]) torch.int64


In [ ]:
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        return self.layers(x)

model = MLP().to(device)
print(model)

Using cpu device
MLP(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(data_loader, model, criterion, optimizer):
    model.train()

    num_batches = len(data_loader)
    num_items = len(data_loader.dataset)

    total_loss = 0
    total_correct = 0
    for data, target in data_loader:
        # Copy data and targets to GPU
        data = data.to(device)
        target = target.to(device)

        # Do a forward pass
        output = model(data)

        # Calculate the loss



        loss = criterion(output, target)
        total_loss += loss

        # Count number of correct digits


        total_correct += (torch.argmax(output,dim = 1)== target).sum().item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_loss = total_loss/num_batches
    accuracy = total_correct/num_items

    print(f"Average loss: {train_loss:7f}, accuracy: {accuracy:.2%}")


In [ ]:
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_dataloader, model, criterion, optimizer)

Training epoch: 1
Average loss: 0.052667, accuracy: 98.37%
Training epoch: 2
Average loss: 0.038320, accuracy: 98.86%
Training epoch: 3
Average loss: 0.031293, accuracy: 99.03%
Training epoch: 4
Average loss: 0.027447, accuracy: 99.15%
Training epoch: 5
Average loss: 0.024470, accuracy: 99.23%
Training epoch: 6
Average loss: 0.020913, accuracy: 99.34%
Training epoch: 7
Average loss: 0.016691, accuracy: 99.49%
Training epoch: 8
Average loss: 0.017230, accuracy: 99.50%
Training epoch: 9
Average loss: 0.014248, accuracy: 99.52%
Training epoch: 10
Average loss: 0.015127, accuracy: 99.59%
